In [1]:
import requests
import pandas as pd
from textblob import TextBlob
from dotenv import load_dotenv

# Load API key from environment variable
load_dotenv()
api_key = "1da70b653f044d708e29b6b5f623a37d"  # Hardcoded for simplicity

# Define the pre-defined categories for investors and politicians
categories = {
    "Stock Market": ["stock market", "stocks", "trading", "investment", "economy"],
    "Politics": ["politics", "government", "election", "policy", "congress"]
}

# Function to perform sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return "Positive"
    elif analysis.sentiment.polarity == 0:
        return "Neutral"
    else:
        return "Negative"

# Fetch news data and create DataFrame
def fetch_news_data():
    news_data = []
    for category, keywords in categories.items():
        for keyword in keywords:
            url = f"https://newsapi.org/v2/everything?q={keyword}&from=2024-03-08&to=2024-03-09&sortBy=publishedAt&apiKey={api_key}"
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                articles = data["articles"]
                for article in articles:
                    news_data.append({
                        "News": article["title"],
                        "Sentiment": get_sentiment(article["title"]),
                        "Category": category
                    })
    return pd.DataFrame(news_data)

# Get news data and display
news_df = fetch_news_data()

news_df

,News,Sentiment,Category
0,"Accor rejoins the CAC 40, the Paris stock exch...",Neutral,Stock Market
1,Marinus Pharmaceuticals (NASDAQ:MRNS) Shares G...,Negative,Stock Market
2,Riskified (NYSE:RSKD) Sees Unusually-High Trad...,Neutral,Stock Market
3,The AI Gold Rush,Neutral,Stock Market
4,Vesuvius Plc - Transaction in Own Shares,Positive,Stock Market
...,...,...,...
995,Biden takes aim at Trump by saying the U.S. is...,Neutral,Politics
996,[Removed],Neutral,Politics
997,‘State of the Union is genocide’: Gaza protest...,Neutral,Politics
998,Group of US Congress members don blue outfits ...,Neutral,Politics


In [6]:
import requests
import pandas as pd
from textblob import TextBlob
from dotenv import load_dotenv
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Load API key from environment variable
load_dotenv()
api_key = "1da70b653f044d708e29b6b5f623a37d"  # Hardcoded for simplicity

# Define the pre-defined categories for investors and politicians
categories = {
    "Stock Market": ["stock market", "stocks", "trading", "investment", "economy"],
    "Politics": ["politics", "government", "election", "policy", "congress"]
}

# Function to perform sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return "Positive"
    elif analysis.sentiment.polarity == 0:
        return "Neutral"
    else:
        return "Negative"

# Function to perform Named Entity Recognition (NER), Dependency Parsing, and Event Extraction
def analyze_article(article_text):
    doc = nlp(article_text)
    entities = ", ".join([ent.text for ent in doc.ents])
    events = ", ".join([token.lemma_ for token in doc if token.pos_ == "VERB"])
    dependencies = ", ".join([token.dep_ for token in doc])
    return entities, events, dependencies

# Fetch news data and create DataFrame
def fetch_news_data():
    news_data = []
    for category, keywords in categories.items():
        for keyword in keywords:
            url = f"https://newsapi.org/v2/everything?q={keyword}&from=2024-03-08&to=2024-03-09&sortBy=publishedAt&apiKey={api_key}"
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                articles = data["articles"]
                for article in articles:
                    entities, events, dependencies = analyze_article(article["content"])
                    news_data.append({
                        "News": article["title"],
                        "Sentiment": get_sentiment(article["title"]),
                        "Category": category,
                        "Entities": entities,
                        "Events": events,
                        "Dependencies": dependencies
                    })
    return pd.DataFrame(news_data)

# Get news data and display
news_df = fetch_news_data()

news_df

,News,Sentiment,Category,Entities,Events,Dependencies
0,Column-UK equity 'dark age' reflects alarming ...,Negative,Stock Market,"Mike Dolan\r\n, LONDON, Reuters, British, UK","try, steer, have, give","ROOT, compound, compound, dep, pobj, punct, ap..."
1,AVI Global Trust (LON:AGT) Trading Up 0.2%,Neutral,Stock Market,"AVI Global Trust, 0.2%, Thursday, 2.92, 228.50...","get, trade, trade","compound, compound, nsubj, punct, appos, punct..."
2,E3 Metals (CVE:ETMC) Trading 1.3% Higher,Positive,Stock Market,"E3 Metals Corp., 1.3%, Thursday, as high as C$...","get, ’, trade, trade, trade, trade","compound, compound, nsubj, punct, appos, punct..."
3,Scottish American (LON:SAIN) Trading Down 0.2%,Negative,Stock Market,"Scottish American, 0.2%, Thursday, 6.30, GBX 4...","get, ’, trade, trade, trade","amod, nsubj, punct, appos, punct, appos, punct..."
4,Taseko Mines Limited - Taseko Reports Strong F...,Positive,Stock Market,"$190 Million, 2023",read,"compound, ROOT, compound, compound, compound, ..."
...,...,...,...,...,...,...
995,Gold Star Father Put in Handcuffs for Protesti...,Negative,Politics,"Gold Star, AfghanistanGateway Pundit, 3-7-2024...","skip, Put, protest","ROOT, prep, pobj, punct, dep, compound, compou..."
996,5 of the biggest moments from Biden's State of...,Neutral,Politics,"Joe Biden's, Thursday, Democratic, Republican,...","fill, make, seek, make","compound, compound, poss, case, amod, nsubjpas..."
997,Biden stares down questions about his age and ...,Positive,Politics,"WASHINGTON, Joe Biden, Thursday, night, Americ...","need, deliver","ROOT, det, nsubj, dobj, compound, nsubj, advmo..."
998,Biden Reaffirms Need to Free American Journali...,Positive,Politics,"Thursday, night, U.S., Joe Biden, Evan Gershko...","reaffirm, secure, jail","prep, poss, pobj, prep, det, compound, pobj, p..."
